In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from scipy import stats

In [ ]:
df = pd.read_pickle('./data/processed/counts_df.pkl')
df.head()

## Exploratory Data Analysis

### Capacity (MWe)

In [ ]:
fig = px.histogram(df['capacity (MWe)'])
fig.update_layout(
    width=750,
    height=500,
    template='plotly_white',
    xaxis_title='Capacity (MWe)',
    yaxis_title='Count',
    showlegend=False,
)
fig.show()

### Scrams

In [ ]:
# Actual Counts
hist_x, hist_y = [], []
for cnt in range(0, max(df['scrams']) + 1):
    hist_x.append(cnt)
    hist_y.append(len(df[df['scrams'] == cnt]))
hist_x, hist_y

# Theoretical Counts
lambda_ = np.mean(df['scrams'])
theory_y = [stats.poisson.pmf(x, mu=lambda_)*len(df) for x in hist_x]

# Figure
fig = go.Figure()
fig.add_trace(
    go.Bar(x=hist_x, y=hist_y, name='Empirical')
)
fig.add_trace(
    go.Bar(x=hist_x, y=theory_y, name=f'Poisson, lambda={lambda_:.2f}')
)
fig.update_layout(
    width=750,
    height=500,
    template='plotly_white',
    xaxis_title='Scrams',
    yaxis_title='Count',
    title='Empirical and Theoretical Count Distributions',
    legend=dict(
        yanchor="top",
        y=1.0,
        xanchor="left",
        x=0.65
    )
)
fig.show()